In [61]:
# pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
sns.set_style('whitegrid')
%matplotlib inline

In [62]:
train_x = pd.read_csv("train.csv")
test_x    = pd.read_csv("test.csv")
train_y = pd.read_csv("trainLabels.csv")

In [63]:
print train_x.shape,train_y.shape,test_x.shape

(999, 40) (999, 1) (8999, 40)


In [64]:
y = np.array(train_y)
train_y = y.reshape(999,)

In [19]:
# start with SVM
from sklearn.svm import SVC
model = SVC()
model.fit(train_x,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
# add Grid Search to find best parama
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid.fit(train_x,train_y)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] kernel=rbf, C=0.1, gamma=1 ......................................
[CV] ....... kernel=rbf, C=0.1, gamma=1, score=0.508982, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=1 ......................................
[CV] ....... kernel=rbf, C=0.1, gamma=1, score=0.510511, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=1 ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ....... kernel=rbf, C=0.1, gamma=1, score=0.509036, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=0.1 ....................................
[CV] ..... kernel=rbf, C=0.1, gamma=0.1, score=0.508982, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=0.1 ....................................
[CV] ..... kernel=rbf, C=0.1, gamma=0.1, score=0.510511, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=0.1 ....................................
[CV] ..... kernel=rbf, C=0.1, gamma=0.1, score=0.509036, total=   0.1s
[CV] kernel=rbf, C=0.1, gamma=0.01 ...................................
[CV] .... kernel=rbf, C=0.1, gamma=0.01, score=0.853293, total=   0.0s
[CV] kernel=rbf, C=0.1, gamma=0.01 ...................................
[CV] .... kernel=rbf, C=0.1, gamma=0.01, score=0.843844, total=   0.0s
[CV] kernel=rbf, C=0.1, gamma=0.01 ...................................
[CV] .... kernel=rbf, C=0.1, gamma=0.01, score=0.810241, total=   0.0s
[CV] kernel=rbf, C=0.1, gamma=0.001 ..................................
[CV] .

[CV] ...... kernel=rbf, C=1000, gamma=1, score=0.508982, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=1 .....................................
[CV] ...... kernel=rbf, C=1000, gamma=1, score=0.510511, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=1 .....................................
[CV] ...... kernel=rbf, C=1000, gamma=1, score=0.509036, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=0.1 ...................................
[CV] .... kernel=rbf, C=1000, gamma=0.1, score=0.517964, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=0.1 ...................................
[CV] .... kernel=rbf, C=1000, gamma=0.1, score=0.534535, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=0.1 ...................................
[CV] .... kernel=rbf, C=1000, gamma=0.1, score=0.524096, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=0.01 ..................................
[CV] ... kernel=rbf, C=1000, gamma=0.01, score=0.901198, total=   0.1s
[CV] kernel=rbf, C=1000, gamma=0.01 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    7.1s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'kernel': ['rbf'], 'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [8]:
grid.best_score_

0.90890890890890896

In [65]:
X_train = train_x
y_train = train_y
print X_train.shape, y_train.shape


(999, 40) (999,)


In [66]:
# using tensorflow MNN

# === We build the graph here!
london_data_science_graph = tf.Graph()

with london_data_science_graph.as_default():

    HIDDEN_SIZE = 200
    num_features = X_train.shape[1]
    
    # create the neural network model
    keep_prob = tf.placeholder(tf.float32)
    prev_loss = tf.Variable(0., trainable=False)
    
    # first layer
    input_layer = tf.placeholder(tf.float32, [None, num_features], name='input')
    W1 = tf.Variable(tf.random_normal([num_features, HIDDEN_SIZE], stddev=.01), name='W1')
    b1 = tf.Variable(tf.random_normal([HIDDEN_SIZE], stddev=.01), name='b1')
    h1_layer = tf.add(tf.matmul(input_layer, W1), b1)
    h1_layer = tf.nn.relu(h1_layer)
    h1_layer = tf.nn.dropout(h1_layer, keep_prob, name='h1')
    
    # second layer
    W2 = tf.Variable(tf.random_normal([HIDDEN_SIZE, HIDDEN_SIZE], stddev=.01), name='W2')
    b2 = tf.Variable(tf.random_normal([HIDDEN_SIZE], stddev=.01), name='b2')
    h2_layer = tf.matmul(h1_layer, W2) + b2
    h2_layer = tf.nn.relu(h2_layer)
    h2_layer = tf.nn.dropout(h2_layer, keep_prob, name='h2')
    
    # third layer, output layer
    W3 = tf.Variable(tf.random_normal([HIDDEN_SIZE, 1], stddev=.01), name='W3')
    b3 = tf.Variable(tf.random_normal([1], stddev=.01), name='b3')
    output_layer = tf.matmul(h2_layer, W3) + b3
    y = tf.placeholder(tf.float32, shape=[None, 1], name='y')
    
    # cost function, optimizer to global minimal
    loss = tf.squared_difference(output_layer, y)
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_constant = 0.01  # Choose an appropriate one.
    loss = loss + reg_constant * sum(reg_losses)
    loss = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-6, beta1=.85, beta2=.9).minimize(loss)
    init = tf.global_variables_initializer()

In [67]:
def get_next_batch(x_data,y_data,batch_size):
    idx = np.random.randint(0, len(x_data), batch_size)
    train_batches_x = x_data[idx]
    train_batches_y = y_data[idx]
    return train_batches_x, train_batches_y

In [68]:
from sklearn.model_selection import train_test_split
X_df = X_train
y_df = y_train
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=101)

In [69]:
X_train_array = X_train.as_matrix()
y_train_array = np.expand_dims(y_train,1)
X_test_array = X_test.as_matrix()
y_test_array = np.expand_dims(y_test,1)

In [70]:
print X_train_array.shape, y_train_array.shape, X_test_array.shape, y_test_array.shape
print type(X_train_array),type(y_train_array),type(X_test_array),type(y_test_array)

(799, 40) (799, 1) (200, 40) (200, 1)
<type 'numpy.ndarray'> <type 'numpy.ndarray'> <type 'numpy.ndarray'> <type 'numpy.ndarray'>


In [72]:
# We create our sessions
sess = tf.Session(graph=london_data_science_graph)

# Make sure to run the initialization
sess.run(init)

NUM_EPOCHS = 100000
BATCH_SIZE = 100
train_loss = []
valid_loss = []

# get the next batch
for i in range(NUM_EPOCHS):
    x_batch, y_batch = get_next_batch(X_train_array,y_train_array,BATCH_SIZE)
    #print x_batch.shape, type(x_batch), y_batch.shape, type(y_batch)
    
    sess.run(optimizer, feed_dict={input_layer: x_batch, y: y_batch, keep_prob: .75})
    
    train_loss.append(sess.run(loss, feed_dict={input_layer: x_batch, y: y_batch, keep_prob: .75}))
    valid_loss.append(sess.run(loss, feed_dict={input_layer: X_test_array, y: y_test_array, keep_prob: 1.}))
    
    if i%1000 == 0:
        print "--------Epochs:{}--------".format(i)
        print "train error:", train_loss[i]
        print "valid error:", valid_loss[i]

#print train_error;
#print "validation error:", sess.run(error, feed_dict={x:test_inputs, y:test_outputs})
sess.close()

--------Epochs:0--------
train error: 0.526509
valid error: 0.551777
--------Epochs:1000--------
train error: 0.484287
valid error: 0.538509
--------Epochs:2000--------
train error: 0.488666
valid error: 0.522607
--------Epochs:3000--------
train error: 0.443776
valid error: 0.501676
--------Epochs:4000--------
train error: 0.457171
valid error: 0.474595
--------Epochs:5000--------
train error: 0.35723
valid error: 0.44141
--------Epochs:6000--------
train error: 0.317716
valid error: 0.402893
--------Epochs:7000--------
train error: 0.317835
valid error: 0.360704
--------Epochs:8000--------
train error: 0.256888
valid error: 0.317536
--------Epochs:9000--------
train error: 0.247083
valid error: 0.277074
--------Epochs:10000--------
train error: 0.249644
valid error: 0.243278
--------Epochs:11000--------
train error: 0.190003
valid error: 0.217796
--------Epochs:12000--------
train error: 0.187396
valid error: 0.197919
--------Epochs:13000--------
train error: 0.169244
valid error: 0.

In [1]:
from sklearn.metrics import classification_report

In [ ]:
#print(classification_report(y_test,predictions))